In [2]:
# Community simulator package
from IPython.display import Image
from community_simulator import *
from community_simulator.usertools import *
from community_simulator.visualization import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends import backend_pdf as bpdf
import numpy as np
import scipy as sp
colors = sns.color_palette()
%matplotlib inline

# Community selection package
from community_selection import *
from community_selection.A_experiment_functions import *
from community_selection.B_community_phenotypes import *
from community_selection.C_selection_algorithms import *
from community_selection.D_migration_algorithms import *

cvxpy not installed. Community.SteadyState() not available.


In [3]:
make_algorithm_library()

,AlgorithmType,AlgorithmName
0,community_phenotype,f1_additive
1,community_phenotype,f2_interaction
2,community_phenotype,f3_additive_binary
3,community_phenotype,f4_interaction_binary
4,community_phenotype,f5_invasion_growth
5,community_phenotype,f6_resident_growth
0,selection_algorithm,no_selection
1,selection_algorithm,select_top25percent
2,selection_algorithm,select_top10percent
3,selection_algorithm,select_bottom25percent


# Main funtion for simulating under one set of algorithms

In [4]:
def simulate_community( 
    assumptions,
    params_simulation,
    params_algorithm,
    file_name = "data/",
    assembly_type = "self_assembly",
    write_composition = False):
    """
    Simulate community dynamics by given experimental regimes
    
    params_simulation = dictionary of parameters for running experiment
    params_algorithm = dictionary of algorithms that determine the selection regime, migration regime, and community pheotypes of interest
    
    Return
    community_composition = concatenated, melted panda dataframe of community and resource composition in each transfer
    community_function = melted panda dataframe of community function
    """
    # Print out the algorithms
    print("\nAlgorithm: "+ params_algorithm["algorithm_name"][0])
    print("\n")
    print(params_algorithm[["transfer", "community_phenotype", "selection_algorithm", "migration_algorithm"]].to_string(index = False))

    # Set seeds
    np.random.seed(2)
    
    # Make initial state
    init_state = MakeInitialState(assumptions)

    # Make plate
    plate = Community(init_state, dynamics, params, scale = 10**6, parallel = True) 
    setattr(plate, "species_function", species_function)
    setattr(plate, "interaction_function", interaction_function)
    
    # Update the community composition by sampling from the pool
    print("\nGenerating initial plate")
    plate.N = sample_from_pool(plate.N, scale = 10**6, inocula = 10**6)
    
    # Empty list for saving data
    plate_data_list = list() # Plate composition
    community_function_list = list() # Community function

    # Save the inocula composition
    plate_data = reshape_plate_data(plate, transfer_loop_index = 0, assembly_type = assembly_type) # Initial state
    plate_data_list.append(plate_data)
    
    # Output the file if write_composition set True
    if write_composition == True:
        plate_data.to_csv(file_name + "-T" + "{:02d}".format(0) + ".txt", index = False)
        
    print("\nStart propogation")
    # Run simulation
    for i in range(0, params_simulation["n_transfer"]):
        # Algorithms used in this transfer
        phenotype_algorithm = params_algorithm["community_phenotype"][i]
        selection_algorithm = params_algorithm["selection_algorithm"][i]
        migration_algorithm = params_algorithm["migration_algorithm"][i]

        # Print the propagation progress
        print("\nTransfer " + str(i+1) +
              "\tphenotype algorithm: " + phenotype_algorithm + 
              "\tselection algorithm: " + selection_algorithm + 
              "\tmigration algorithm: " + migration_algorithm)
        
        # Propagation
        plate.Propagate(params_simulation["n_propagation"])
    
        # Append the composition to a list
        plate_data = reshape_plate_data(plate, transfer_loop_index = (i + 1), assembly_type = assembly_type) # Transfer = 0 means that it's before selection regime works upon
        plate_data_list.append(plate_data)

        ## Output the file if write_composition set True
        if write_composition == True:
            plate_data.to_csv(file_name + "-T" + "{:02d}".format(i + 1) + ".txt", index = False) # Transfer = 0 means that it's before selection regime works upon

        # Community phenotype, richness, and biomass
        community_function = globals()[phenotype_algorithm](plate, assumptions = assumptions) # Community phenotype
        richness = np.sum(plate.N > 0, axis = 0) # Richness
        biomass = list(np.sum(plate.N, axis = 0)) # Biomass
        community_function_list.append(reshape_function_data(community_function, richness, biomass, transfer_loop_index = i)) # Transfer = 0 means that it's before selection regime works upon

        # Passage and tranfer matrix
        transfer_matrix = globals()[selection_algorithm](community_function)
        plate.Passage(transfer_matrix * params_simulation["dilution"])
        
        # Migration
        m = globals()[migration_algorithm](community_function) 
        plate.N = migrate_from_pool(plate, params_simulation["pool"], m)
        
    print("\nAlgorithm "+ params_algorithm["algorithm_name"][0] + " finished")

    # Concatenate data from from different transfers
    plate_data_con = pd.concat(plate_data_list)
    community_function_con = pd.concat(community_function_list)
    
    

    return plate_data_con, community_function_con

Parameters that determine the universe

In [5]:
# Make dynanmics by default we will use the microbial consumer resource model
def dNdt(N,R,params):
    return MakeConsumerDynamics(assumptions)(N,R,params)
def dRdt(N,R,params):
    return MakeResourceDynamics(assumptions)(N,R,params)
dynamics = [dNdt,dRdt]

# Universal parameters
assumptions = a_default.copy() # Start with default parameters
assumptions = {
    'sampling':'Binary', #{'Gaussian','Binary','Gamma'} specifies choice of sampling algorithm    
    'SA': 60*np.ones(3), #Number of species in each specialist family (here, 3 families of 60 species)
    'MA': 30*np.ones(3), #Number of resources in each class 
    'Sgen': 30, #Number of generalist species (unbiased sampling over alll resource classes)
    'muc': 10, #Mean sum of consumption rates (used in all models)
    'sigc': 3, #Standard deviation of sum of consumption rates for Gaussian and Gamma models
    'q': 0.0, #Preference strength of specialist families (0 for generalist and 1 for specialist)
    'c0':0.0, #Sum of background consumption rates in binary model
    'c1':1, #Specific consumption rate in binary model
    'l':0.8, #Leakage fraction
    'fs':0.45, #Fraction of secretion flux with same resource type
    'fw':0.45, #Fraction of secretion flux to 'waste' resource
    'sparsity':0.2, #Effective sparsity of metabolic matrix (between 0 and 1)
    'n_wells':24, #Number of independent wells
    'S':100, #Number of species per well (randomly sampled from the pool of size Stot = sum(SA) + Sgen)
    'food':0, #index of food source (when a single resource is supplied externally)
    'R0_food':1000, #unperturbed fixed point for supplied food
    'regulation':'independent', #metabolic regulation (see dRdt)
    'response':'type III', #functional response (see dRdt)
    'supply':'off' #resource supply (see dRdt)
}

## Parameters not included in the community-simulator package
assumptions.update({
    "m": 0, # Mortality
    "sigma" : 1, # Standard deviation for drawing specifc speices/interaction function
    "alpha": 1 # Scaling factor between species- and interaction-specific function variances
})

# Prepare experiment setup in this universe
params, species_pool, species_function, interaction_function = prepare_experiment(assumptions, seed = 1)

## Simulation parameters
params_simulation = {
    "n_propagation": 24, # Length of propagation, or hours within a growth cycle
    "n_transfer": 10, # Number of transfer, or number of passage
    "dilution": 1/125, # Dilution factor at every transfer
    "pool": species_pool 
}

In [6]:
S_tot = 210
inocula = 10**6
# For each well, sample community from different microbiome sample
np.random.seed(2) 
pool = np.random.power(1, size  = S_tot) # Power-law distribution
pool = pool/np.sum(pool) # Normalize the pool
consumer_list = np.random.choice(len(pool), size = inocula, replace = True, p = pool) # Draw from the pool


In [11]:
my_tab = pd.crosstab(index = consumer_list, columns = "count")
np.ravel(my_tab)/10**6

array([4.3930e-03, 2.6300e-04, 5.6910e-03, 4.4150e-03, 4.1660e-03,
       3.3280e-03, 2.1280e-03, 6.2270e-03, 3.1810e-03, 2.6640e-03,
       6.3250e-03, 5.5290e-03, 1.3310e-03, 5.1990e-03, 1.8970e-03,
       8.0070e-03, 8.5610e-03, 5.1410e-03, 8.6810e-03, 8.0400e-04,
       5.1870e-03, 6.3600e-04, 4.3910e-03, 9.8500e-04, 1.3030e-03,
       6.0450e-03, 2.2320e-03, 1.0850e-03, 2.1700e-03, 3.5620e-03,
       4.7160e-03, 2.0950e-03, 6.4990e-03, 4.8620e-03, 5.0240e-03,
       4.1170e-03, 8.1480e-03, 5.8530e-03, 1.6570e-03, 7.0770e-03,
       9.8530e-03, 4.9420e-03, 9.0310e-03, 3.4280e-03, 5.7580e-03,
       4.3650e-03, 4.4450e-03, 7.9330e-03, 5.4580e-03, 9.7190e-03,
       5.3940e-03, 8.4400e-04, 3.7390e-03, 8.5500e-03, 4.1910e-03,
       2.6400e-04, 2.5330e-03, 6.8900e-04, 1.0071e-02, 9.7920e-03,
       8.2000e-03, 6.1680e-03, 7.6580e-03, 1.7230e-03, 2.9970e-03,
       5.3090e-03, 3.6030e-03, 4.7200e-04, 9.9760e-03, 4.4640e-03,
       4.9720e-03, 3.3300e-03, 2.5300e-03, 3.8590e-03, 8.4480e

In [5]:
simple_screening = pd.DataFrame({
    "algorithm_name": "simple_screening",
    "transfer": range(1, params_simulation["n_transfer"] + 1),
    "community_phenotype": ["f1_additive" for i in range(params_simulation["n_transfer"])], 
    "selection_algorithm": ["no_selection" for i in range(params_simulation["n_transfer"])], 
    "migration_algorithm": ["no_migration" for i in range(params_simulation["n_transfer"])]
})

screening_plate_df, screening_function_df = simulate_community(
    assumptions = assumptions,
    params_simulation = params_simulation, 
    params_algorithm = simple_screening, 
    write_composition = False
)


Algorithm: simple_screening


 transfer community_phenotype selection_algorithm migration_algorithm
        1         f1_additive        no_selection        no_migration
        2         f1_additive        no_selection        no_migration
        3         f1_additive        no_selection        no_migration
        4         f1_additive        no_selection        no_migration
        5         f1_additive        no_selection        no_migration
        6         f1_additive        no_selection        no_migration
        7         f1_additive        no_selection        no_migration
        8         f1_additive        no_selection        no_migration
        9         f1_additive        no_selection        no_migration
       10         f1_additive        no_selection        no_migration

Generating initial plate

Start propogation

Transfer 1	phenotype algorithm: f1_additive	selection algorithm: no_selection	migration algorithm: no_migration

Transfer 2	phenotype algorithm: f1_addit

In [6]:
screening_plate_df.to_csv("data/simple_screening.txt")
#screen_plate_df[screen_plate_df]

# Example selection algorithms

The dictionary `params_algorithm` takes three lists:

- community_phenotype
- selection_algorithm
- migration_algorithm

The lengh of each algorithm list must have the same number of experimental transfers

In [ ]:
# Test algorithm that the communities are simply passaged
simple_screening = pd.DataFrame({
    "algorithm_name": "simple_screening",
    "transfer": range(1, params_simulation["n_transfer"] + 1),
    "community_phenotype": "f1_additive",
    "selection_algorithm": "no_selection",
    "migration_algorithm": "no_migration"
})

# Direction selection
direct_selection = pd.DataFrame({
    "algorithm_name": "direct_selection",
    "transfer": range(1, params_simulation["n_transfer"] + 1),
    "community_phenotype": "f1_additive", 
    "selection_algorithm": ["no_selection" for i in range(10)] + ["direct_selection_select"] + ["no_selection" for i in range(params_simulation["n_transfer"] - 11)], 
    "migration_algorithm": ["no_migration" for i in range(10)] + ["direct_selection_migrate"] + ["no_migration" for i in range(params_simulation["n_transfer"] - 11)]
})

# Select top 25%
select_top25percent = pd.DataFrame({
    "algorithm_name": "select_top25percent",
    "transfer": range(1, params_simulation["n_transfer"] + 1),
    "community_phenotype": "f1_additive",
    "selection_algorithm": ["no_selection" for i in range(10)] + ["select_top25percent"] + ["no_selection" for i in range(params_simulation["n_transfer"] - 11)], 
    "migration_algorithm": "no_migration"
})


## Simple screening

In [ ]:
screening_plate_df, screening_function_df = simulate_community(
    assumptions = assumptions,
    params_simulation = params_simulation, 
    params_algorithm = simple_screening, 
    write_composition = False
)

In [ ]:
plot_community_function(screening_function_df)

## Direct selection algorithm

In [ ]:
direct_selection_plate_df, direct_selection_function_df = simulate_community(
    assumptions = assumptions,
    params_simulation = params_simulation, 
    params_algorithm = direct_selection, 
    write_composition = False
)

In [ ]:
plot_community_function(direct_selection_function_df)

## Select the top 25%

In [ ]:
select_top25percent_plate_df, select_top25percent_function_df = simulate_community(
    assumptions = assumptions,
    params_simulation = params_simulation, 
    params_algorithm = select_top25percent, 
    write_composition = False
)

In [ ]:
plot_community_function(select_top25percent_function_df)

In [18]:
inocula = 10**8
np.random.seed(1)
pool = np.random.power(1, size = 210)
pool = pool / np.sum(pool)
np.random.choice(len(pool), size = inocula, replace = True, p = pool) # Draw from the pool


array([ 85, 204,  39, ..., 138,  80, 132])